# Using ChatGPT or Google Gemini with Python for real-world tasks

In this notebook you will use the OpenAI's ChatGPT or Google Gemini to solve:

- Exercise - 1: Review Analysis and Response
- Exercise - 2: Paper Analysis and Summarization
- Exercise 3 - Marketing Adverts for Smartphones
- Exercise 4 - IT Support Resolution

___[Created By: Dipanjan (DJ)](https://www.linkedin.com/in/dipanjans/)___

## Install OpenAI dependencies

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


## Optional: Install Google Gemini

Google Gemini API is free (till now). You can get a key [here](https://aistudio.google.com/app/apikey), just need to sign in with your google account. Gemini may not be available fully in EU.

In [ ]:
pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.6/663.6 kB 9.7 MB/s eta 0:00:00


## Load OpenAI API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import yaml

with open('chatgpt_api_credentials.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [ ]:
api_creds.keys()

dict_keys(['openai_key'])

In [ ]:
import openai
from IPython.display import HTML

openai.api_key = api_creds['openai_key']

## Load Gemini API credentials

Run this section only if you are using Google Gemini

In [ ]:
import google.generativeai as genai
import yaml

with open('gemini_key.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

GOOGLE_API_KEY = api_creds['gemini_key']
genai.configure(api_key=GOOGLE_API_KEY)

## Create ChatGPT and Google Gemini Chat Completion Access Function

This function will use the [Chat Completion API](https://platform.openai.com/docs/api-reference/chat/create) to access ChatGPT for us and return responses

We also add the capability to access the [Google Gemini API](https://ai.google.dev/tutorials/python_quickstart) if you want to use Gemini Pro instead of ChatGPT

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
  if model == "gpt-3.5-turbo":
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # degree of randomness of the model's output
    )
    return response.choices[0].message.content
  if model == 'gemini-pro':
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    return response.text
  else:
    return "LLM not configured! Please configure logic for specific model in get_completion()"

## Let's try out the ChatGPT API!

In [ ]:
response = get_completion(prompt='Explain Generative AI in 2 bullet points')
print(response)

- Generative AI is a type of artificial intelligence that is capable of creating new content, such as images, text, or music, based on patterns and data it has been trained on.
- This technology can be used in a variety of applications, including creating realistic images for video games, generating personalized content for marketing campaigns, or even composing music or writing stories.


In [ ]:
# with Gemini
response = get_completion(prompt='Explain Generative AI in 2 bullet points',
                          model='gemini-pro')
print(response)

* **Generative AI** is a type of artificial intelligence that can create new data or content from scratch, such as generating text, images, music, or code.


* Generative AI uses **machine learning** algorithms to learn the patterns and structures of existing data, and then uses this knowledge to generate new data that is similar to the original but not identical.


## Exercise - 1: Review Analysis and Response

For each review get ChatGPT to do the following:

            - Summarize the following reviews below, delimited by triple
            backticks. For each review, the summary should be at most 3 lines.
            - Highlight both the positives and negatives for each review
            - Display the overall sentiment for each review (positive, negative, neutral)
            - Display a list of 3 - 5 emotions expressed by the customer
            - If the sentiment is positive or neutral write an email and thank them for the review
            - If the sentiment is negative apologize and write an email with an appropriate response

In [ ]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

In [ ]:
responses = []

for review in reviews:
  prompt = f"""
              Act as a product review analyst.
              Your task is to perform the following tasks:

              - Summarize the following review below, delimited by triple
              backticks in at most 3 lines.
              - Highlight both the positives and negatives for the review
              - Display the overall sentiment for the review (positive, negative, neutral)
              - Display a list of 3 - 5 emotions expressed by the customer
              - If the sentiment is positive or neutral write an email and thank them for the review
              - If the sentiment is negative apologize and write an email with an appropriate response

              ```{review}```
              """
  response = get_completion(prompt)
  responses.append(response)

In [ ]:
for response in responses:
  print(response)
  print('------')
  print('\n')

- Positive: Fantastic sound quality, waterproof, compact yet powerful volume, bargain price, early shipping
- Negative: None mentioned
- Overall sentiment: Positive
- Emotions: Satisfaction, excitement, gratitude

Dear [Customer],

Thank you so much for your positive review of the Bluetooth speaker you purchased for beach outings! We are thrilled to hear that you are enjoying the fantastic sound quality, waterproof feature, and compact yet powerful volume. We are also glad that the shipping was on point and arrived early. Your satisfaction is our top priority, and we look forward to serving you again in the future.

Warm regards,
[Your Name]
------


- Positive: Responsive keys, vibrant LED colors, competitive price, swift delivery, well-protected packaging.
- Negative: None mentioned.
- Overall sentiment: Positive
- Emotions: Satisfaction, excitement, gratitude

Dear [Customer],

Thank you for taking the time to leave such a positive review about your new gaming keyboard! We are thril

In [ ]:
# Trying with Gemini

responses = []

for review in reviews:
  prompt = f"""
              Act as a product review analyst.
              Your task is to perform the following tasks:

              - Summarize the following review below, delimited by triple
              backticks in at most 3 lines.
              - Highlight both the positives and negatives for the review
              - Display the overall sentiment for the review (positive, negative, neutral)
              - Display a list of 3 - 5 emotions expressed by the customer
              - If the sentiment is positive or neutral write an email and thank them for the review
              - If the sentiment is negative apologize and write an email with an appropriate response

              ```{review}```
              """
  response = get_completion(prompt, model='gemini-pro')
  responses.append(response)

In [ ]:
for response in responses:
  print(response)
  print('------')
  print('\n')

### Review Summary

**Positives:**

- Excellent sound quality
- Waterproof design
- Compact size with impressive volume
- Affordable price
- Fast and secure shipping

**Negatives:**

- None mentioned

### Overall Sentiment

Positive

### Emotions Expressed by the Customer

- Joy
- Satisfaction
- Surprise
- Contentment
- Appreciation

### Email Response to Customer

Dear [Customer Name],

Thank you so much for taking the time to leave us such a glowing review! We're thrilled that you love your new Bluetooth speaker and that it's exceeding your expectations. We appreciate your feedback and will continue striving to provide top-notch products and services. Your satisfaction is our priority.

Sincerely,
[Your Company Name]
------


***
**Summary:** The customer is extremely satisfied with their new gaming keyboard, praising its responsiveness, backlighting, competitive pricing, swift delivery, and protective packaging.

**Positives:**
- Responsive keys
- Vibrant LED colors
- Competitive pr

## Exercise - 2: Paper Analysis and Summarization

- Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into a short concise version of maximum 10 lines for your audience.

- Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into an executive summary for a healthcare company.
Have bullet points for pros and cons of ethics in Generative AI as mentioned in the paper.

- Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into an executive summary for a generative AI company solving healthcare problems.
Have bullet points for key points mentioned for
Generative AI for text, images and structured data based healthcare

In [ ]:
paper_abstract = f"""
The widespread use of ChatGPT and other emerging technology powered by generative
artificial intelligence (AI) has drawn much attention to potential ethical issues, especially in
high-stakes applications such as healthcare.1–3 However, less clear is how to resolve such
issues beyond following guidelines and regulations that are still under discussion and
development. On the other hand, other types of generative AI have been used to synthesize
images and other types of data for research and practical purposes, which have resolved some
ethical issues and exposed other ethical issues,4,5 but such technology is less often the focus
of ongoing ethical discussions. Here we highlight gaps in current ethical discussions of
generative AI via a systematic scoping review of relevant existing research in healthcare, and
reduce the gaps by proposing an ethics checklist for comprehensive assessment and
transparent documentation of ethical discussions in generative AI development. While the
checklist can be readily integrated into the current peer review and publication system to
enhance generative AI research, it may also be used in broader settings to disclose ethicsrelated considerations in generative AI-powered products (or real-life applications of such
products) to help users establish reasonable trust in their capabilities.

Current ethical discussions on generative AI in healthcare
We conducted a systematic scoping review to analyse current ethical discussions on
generative AI in healthcare. Our search in four major academic research databases for
relevant publications from January 2013 to July 2023 yielded 2859 articles (see Methods for
detailed search strategy and Supplementary Figure S1 for the PRISMA flow diagram), of
which 193 articles were included for analysis based on application data modality (text, image,
or structured data), ethical issues discussed, generative AI involved, and whether generative
AI causes or offers technical solutions for issues raised.

Generative AI for text data-based healthcare
Forty-one of the 193 articles discussed ethical considerations pertaining to generative AI
applications for text data, with 20 articles describing methodological developments or
applications of generative AI and the other 21 articles describing review-type works on this
topic. Although some of these review-type articles used the general term “generative AI”, the
main body and supporting evidence focused on LLMs. Twenty-nine articles had in-depth
discussions on ethical issues, whereas the other 12 articles only briefly touched on some
ethical aspects.
Among the 41 articles, 29 articles focused on discussing ethical issues caused by LLMs (and
specifically by GPT in 16 of the articles), covering a wide range of application scenarios and
considered the application of all 10 ethical principles identified in the review (see Figure 1),
as well as other less discussed concerns such as human-AI interaction, and the rights of
LLMs to be considered as co-authors in scientific papers. One paper only commented briefly
on the need for ethical considerations in LLMs and is summarised in the “Others” category.
Although all ethical principles are equally important, some are discussed more often than
others, e.g., non-maleficence (also referred to in the literature as ‘benevolence’), equity, and
privacy.
Fifteen of the 41 articles aimed to resolve some existing ethical issues (for example,
confidentiality of medical data) by using LLMs and other generative AI (e.g., GAN,
autoencoder or diffusion), such as, to reduce privacy concerns by generating synthetic
medical text, to reduce disparity by providing accessible services and assistance, to detect
health-related misinformation, to generate trusted content, and to improve accountability or
transparency over existing approaches. While most articles focused on either identifying
ethical issues caused by generative AI or proposing generative AI-based solutions, three
articles discussed both to provide a more balanced perspective.

Generative AI for image and structured data-based healthcare
Unlike the diverse application scenarios of generative AI based on text data, for image and
structured data, this use of generative AI focuses on data synthesis and encryption. Hence the
majority of articles discussed the methodological developments of generative AI as giving
rise to a more distinctive and focused set of ethical issues.
5
Notably, of the 98 articles on image data and 58 articles on structured data, more than half
(n=63 for image data and n=33 for structured data) only mentioned ethical considerations as a
brief motivation for methodological developments or as a general discussion point. The rest
included more in-depth discussions or evaluations of ethical issues. Among these 155 articles
(as one article covered multiple modalities), 11 articles were review-type work, where 10
articles reviewed methods that mentioned one or two ethical perspectives, and only one
article24 discussed detailed ethical concerns on generative AI applications.
Resolving privacy issues was the main aim of articles for these two data modalities (n=74 for
image data and n=50 for structured data; see Figure 1), predominantly by generating synthetic
data using GAN. Eight articles on image data and 9 articles on structured data used
generative AI to reduce bias, e.g., by synthesizing data for under-represented subgroups in
existing databases. For both data modalities, we did not see explicit discussions on resolving
autonomy, integrity, or morality issues using generative AI, and for structured data the articles
additionally lacked discussions on trust or transparency.
Only 11 articles for image data selectively discussed some ethical issues that generative AI
can give rise to, without specific discussions regarding autonomy, integrity, or morality. For
structured data, only 4 articles discussed equity, privacy, or data security issues caused by
generative AI. Only two articles on structured data included both the cause and resolving
perspectives by discussing ethical issues that may arise from limitations of methods
proposed, specifically bias induced when synthesizing data in order to resolve privacy issues.
"""

In [ ]:
prompt = f"""
Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into a short concise version of maximum 10 lines for your audience.


Abstract:
```{paper_abstract}```
"""
response = get_completion(prompt)
response

'Generative AI, like ChatGPT, raises ethical concerns in healthcare. Current discussions focus on guidelines and regulations, but gaps exist. A systematic review highlights ethical issues and proposes an ethics checklist for generative AI development. In healthcare, text data applications often discuss ethical considerations, with some focusing on resolving existing issues. Image and structured data applications mainly aim to synthesize data and address privacy concerns. While some articles propose solutions, others only mention ethical considerations briefly. Overall, there is a need for more in-depth discussions on ethical issues in generative AI applications across different data modalities.'

In [ ]:
prompt = f"""
Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into an executive summary for a healthcare company.
Have bullet points for pros and cons of ethics in Generative AI as mentioned in the paper.


Abstract:
```{paper_abstract}```
"""
response = get_completion(prompt)
print(response)

Executive Summary:

The use of generative artificial intelligence (AI) in healthcare, particularly in high-stakes applications, has raised ethical concerns that need to be addressed. This paper highlights the gaps in current ethical discussions surrounding generative AI and proposes an ethics checklist for comprehensive assessment and transparent documentation.

Pros of Ethics in Generative AI:
- Resolving ethical issues such as confidentiality of medical data
- Reducing privacy concerns by generating synthetic medical text
- Providing accessible services and assistance to reduce disparity
- Detecting health-related misinformation
- Improving accountability and transparency over existing approaches

Cons of Ethics in Generative AI:
- Lack of in-depth discussions on resolving autonomy, integrity, or morality issues
- Limited focus on trust and transparency in structured data applications
- Potential bias induced when synthesizing data to resolve privacy issues
- Insufficient considerati

In [ ]:
prompt = f"""
Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into an executive summary for a generative AI company solving healthcare problems.
Have bullet points for key points mentioned for
Generative AI for text, images and structured data based healthcare


Abstract:
```{paper_abstract}```
"""
response = get_completion(prompt)
print(response)


Executive Summary:

Key Points for Generative AI for Text, Images, and Structured Data Based Healthcare:

- Generative AI, such as ChatGPT, is raising ethical concerns in high-stakes applications like healthcare.
- Current ethical discussions in healthcare highlight gaps in addressing ethical issues related to generative AI.
- A systematic scoping review of 2859 articles identified ethical considerations in generative AI applications for text, image, and structured data in healthcare.
- Ethical issues discussed include non-maleficence, equity, privacy, human-AI interaction, and rights of AI models.
- Generative AI is used to resolve ethical issues like confidentiality, privacy concerns, health misinformation, and improving accountability in healthcare.
- Image and structured data-based generative AI focus on data synthesis and encryption, with privacy being a key concern.
- Synthetic data generation using GANs is used to address privacy and bias issues in image and structured data.
- 

## Exercise 3 - Marketing Adverts for Smartphone

You have the technical fact sheets of one smartphone. Try some iterative prompt engineering and do the following:

1. Generate marketing product description for the smartphone

2. Custom product description which has the following:

```
The description should follow this format:

Product Name: <Name of the smartphone>
​
Description: <Brief Overview of the features>
​
Product Specifications:
<Table with key product feature specifications>
​
The description should focus on the most important features
a customer might look for in a phone including the foldable display screen,
processing power, RAM, camera and battery life.
​
After the description, the table should have the
key specifications of the product. It should have two columns.
The first column should have 'Feature'
and the second column should have 'Specification'
and try to put exact numeric values for features if they exist.
Only put the most important features in the table which matter to users.
```

3. Custom product description focusing on specific aspects like display, camera and in less than 60 words

In [ ]:
fact_sheet_mobile = """
PRODUCT NAME
Samsung Galaxy Z Fold4 5G Black
​
PRODUCT OVERVIEW
Stands out. Stands up. Unfolds.
The Galaxy Z Fold4 does a lot in one hand with its 15.73 cm(6.2-inch) Cover Screen.
Unfolded, the 19.21 cm(7.6-inch) Main Screen lets you really get into the zone.
Pushed-back bezels and the Under Display Camera means there's more screen
and no black dot getting between you and the breathtaking Infinity Flex Display.
Do more than more with Multi View. Whether toggling between texts or catching up
on emails, take full advantage of the expansive Main Screen with Multi View.
PC-like power thanks to Qualcomm Snapdragon 8+ Gen 1 processor in your pocket,
transforms apps optimized with One UI to give you menus and more in a glance
New Taskbar for PC-like multitasking. Wipe out tasks in fewer taps. Add
apps to the Taskbar for quick navigation and bouncing between windows when
you're in the groove.4 And with App Pair, one tap launches up to three apps,
all sharing one super-productive screen
Our toughest Samsung Galaxy foldables ever. From the inside out,
Galaxy Z Fold4 is made with materials that are not only stunning,
but stand up to life's bumps and fumbles. The front and rear panels,
made with exclusive Corning Gorilla Glass Victus+, are ready to resist
sneaky scrapes and scratches. With our toughest aluminum frame made with
Armor Aluminum, this is one durable smartphone.
World’s first water resistant foldable smartphones. Be adventurous, rain
or shine. You don't have to sweat the forecast when you've got one of the
world's first water-resistant foldable smartphones.
​
PRODUCT SPECS
OS - Android 12.0
RAM - 12 GB
Product Dimensions - 15.5 x 13 x 0.6 cm; 263 Grams
Batteries - 2 Lithium Ion batteries required. (included)
Item model number - SM-F936BZKDINU_5
Wireless communication technologies - Cellular
Connectivity technologies - Bluetooth, Wi-Fi, USB, NFC
GPS - True
Special features - Fast Charging Support, Dual SIM, Wireless Charging, Built-In GPS, Water Resistant
Other display features - Wireless
Device interface - primary - Touchscreen
Resolution - 2176x1812
Other camera features - Rear, Front
Form factor - Foldable Screen
Colour - Phantom Black
Battery Power Rating - 4400
Whats in the box - SIM Tray Ejector, USB Cable
Manufacturer - Samsung India pvt Ltd
Country of Origin - China
Item Weight - 263 g
"""

In [ ]:
prompt = f"""
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
in triple backticks.
​
Write a brief product description

Technical specifications:
```{fact_sheet_mobile}```
"""
response = get_completion(prompt)
print(response)

Introducing the Samsung Galaxy Z Fold4 5G Black - the ultimate smartphone that stands out, stands up, and unfolds to reveal a world of possibilities. With a stunning 6.2-inch Cover Screen and a massive 7.6-inch Main Screen, you can truly immerse yourself in your favorite content like never before.

Powered by the Qualcomm Snapdragon 8+ Gen 1 processor and 12GB of RAM, this smartphone offers PC-like power in the palm of your hand. The new Taskbar feature allows for seamless multitasking, while the App Pair function lets you launch up to three apps simultaneously on one screen.

But that's not all - the Galaxy Z Fold4 is our toughest foldable smartphone yet, featuring Corning Gorilla Glass Victus+ panels and an Armor Aluminum frame for durability you can rely on. Plus, it's the world's first water-resistant foldable smartphone, so you can take it with you wherever you go, rain or shine.

Experience the future of smartphones with the Samsung Galaxy Z Fold4 5G Black. Get yours today and un

In [ ]:
prompt = f"""
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
in triple backticks.
​
Write a brief product description.

The description should follow this format:

Product Name: <Name of the smartphone>
​
Description: <Brief Overview of the features>
​
Product Specifications:
<Table with key product feature specifications>
​
The description should only focus on the most important features
a customer might look for in a phone including the foldable display screen,
processing power, RAM, camera and battery life.
​
After the description, the table should have the
key specifications of the product. It should have two columns.
The first column should have 'Feature'
and the second column should have 'Specification'
and try to put exact numeric values for features if they exist.
Only put the most important features in the table which matter to users.

Technical specifications:
```{fact_sheet_mobile}```
"""

response = get_completion(prompt)
print(response)

Product Name: Samsung Galaxy Z Fold4 5G Black

Description: The Samsung Galaxy Z Fold4 5G Black is a revolutionary foldable smartphone that combines cutting-edge technology with a sleek design. With a 6.2-inch Cover Screen and a 7.6-inch Main Screen, you can easily switch between tasks and enjoy a truly immersive viewing experience. Powered by a Qualcomm Snapdragon 8+ Gen 1 processor and 12GB of RAM, this phone offers PC-like power in the palm of your hand. The Galaxy Z Fold4 is also our toughest foldable smartphone yet, featuring Corning Gorilla Glass Victus+ and water-resistant technology.

Product Specifications:
| Feature              | Specification                  |
|----------------------|--------------------------------|
| OS                   | Android 12.0                   |
| RAM                  | 12 GB                          |
| Display Size         | 6.2-inch Cover Screen, 7.6-inch Main Screen |
| Processor            | Qualcomm Snapdragon 8+ Gen 1   |
| Storage      

In [ ]:
prompt = f"""
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
in triple backticks.

Write a catchy product description which uses at most 60 words
and focuses on the most important things about the smartphone
which might matter to users like display and camera

Technical specifications:
```{fact_sheet_mobile}```
"""

response = get_completion(prompt)
print(response)

Introducing the Samsung Galaxy Z Fold4 5G Black - the ultimate smartphone experience. Unfold the 7.6-inch Main Screen for immersive viewing. Capture stunning photos with the Under Display Camera. Enjoy PC-like power with Qualcomm Snapdragon 8+ Gen 1 processor. With water resistance and durable materials, this foldable smartphone is ready for anything. Get yours today and elevate your mobile experience.


In [ ]:
response

'Introducing the Samsung Galaxy Z Fold4 5G Black - the ultimate smartphone experience. Unfold the 7.6-inch Main Screen for immersive viewing. Capture stunning photos with the Under Display Camera. Enjoy PC-like power with Qualcomm Snapdragon 8+ Gen 1 processor. With water resistance and durable materials, this foldable smartphone is ready for anything. Get yours today and elevate your mobile experience.'

## Exercise 4 - IT Support Resolution

Ask ChatGPT to act as a customer support agent, process each customer ticket message and output the response in JSON with the following fields
```
orig_msg: The original customer message
orig_lang: Detected language of the customer message e.g. Spanish
category: 1-2 word describing the category of the problem
trans_msg: Translated customer message in English
response: Response to the customer in orig_lang
trans_response: Response to the customer in English
```

Hint: Use the following prompt in a for loop

```
prompt = f"""
            Act as a customer support agent. For the IT support message mentioned below
            in triple backticks, create a response is JSON in the following format:

            {{
                orig_msg: The original customer message
                orig_lang: Detected language of the customer message e.g. Spanish
                category: 1-2 word describing the category of the problem
                trans_msg: Translated customer message in English
                response: Response to the customer in orig_lang
                trans_response: Response to the customer in English
            }}

            Message:
            '''{msg}'''
            """
```

In [ ]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

In [ ]:
responses = []

for msg in it_support_queue:
  prompt = f"""
            Act as a customer support agent. For the IT support message mentioned below
            in triple backticks, create a response is JSON in the following format:

            {{
                orig_msg: The original customer message
                orig_lang: Detected language of the customer message e.g. Spanish
                category: 1-2 word describing the category of the problem
                trans_msg: Translated customer message in English
                response: Response to the customer in orig_lang
                trans_response: Response to the customer in English
            }}

            Message:
            '''{msg}'''
            """
  response = get_completion(prompt)
  responses.append(response)

In [ ]:
for response in responses:
  print(response)
  print('-----')
  print('\n')

{
    orig_msg: "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha."
    orig_lang: "Portuguese"
    category: "Sync Issue"
    trans_msg: "I can't sync my contacts with the phone. I always get a failure message."
    response: "Olá, lamentamos o inconveniente que está enfrentando. Para resolver esse problema, recomendamos que verifique a conexão com a internet e as configurações de sincronização no seu telefone. Se o problema persistir, entre em contato conosco para obter assistência adicional."
    trans_response: "Hello, we apologize for the inconvenience you are facing. To resolve this issue, we recommend checking the internet connection and sync settings on your phone. If the problem persists, please contact us for further assistance."
}
-----


{
    orig_msg: "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete."
    orig_lang: "Italian"
    category: "Printing"
    trans_msg: "I am having troub